Original https://ts.gluon.ai/stable/tutorials/forecasting/quick_start_tutorial.html

In [1]:
!pip install --upgrade mxnet
!pip install gluonts
!pip install orjson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 9.2 MB/s eta 0:00:00


In [2]:
import gluonts, orjson
#from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.mx import DeepAREstimator, Trainer
import numpy as np
import pandas as pd
from gluonts.dataset.common import ListDataset
#from gluonts.dataset.pandas import PandasDataset
#from gluonts.dataset.split import split

Целевые данные без признаков

In [6]:
def new_ds(df: pd.DataFrame) -> list:
  '''Конвертирует pandas dataframe в датасет.
  Входной параметр: df - датафрейм следующего формата: индекс - дата,
    item_id - обязательная колонка, содержащая идентификатор серии,
    target  - обязательная колонка, содержащая целевое значение.
  Возвращает датасет в виде списка.'''
  ls_res = [{'target': df.loc[df.item_id == item, 'target'].tolist(),
             'start': df.index[0]} for item in df.item_id.unique()]
  return ListDataset(ls_res, freq=freq)

In [7]:
# Загружаем данные
df = pd.read_csv('/content/drive/MyDrive/trainee/Ingate/Ozon/data_ozon.csv',
                 parse_dates=['День'])
# Переименовываем длинные названия с пробелами и кириллическими буквами
dic_ren = {'День': 'timestamp', 'Ozon ID': 'item_id',
           'Заказано товаров': 'target'}
           #'Уникальные посетители, всего': 'dyn_real_1',
           #'Показы, всего': 'dyn_real_2',
           #'Показы в поиске и каталоге': 'dyn_real_3'}
df = df.rename(columns=dic_ren)
freq = 'D'
# Отбираем только нужные столбцы
df = df[['timestamp', 'item_id', 'target']]
          #'dyn_real_1', 'dyn_real_2', 'dyn_real_3']]
# Отбираем товары с большим количеством заказов
df_tmp = df.groupby('item_id').target.sum()
ls_work = df_tmp[df_tmp > 100].index.tolist()
df_tmp = df[df.item_id.isin(ls_work)]
# Отбираем записи с ненулевыми заказами
df = df_tmp[df_tmp.target > 0]
# Выравниваем все товары по датам, пустоту заполняем нулями
df = df.pivot(index='item_id', columns='timestamp', values='target')
df.fillna(0, inplace=True)
df.reset_index(inplace=True)
df = df.melt(('item_id'), var_name='timestamp', value_name='target')
# Сортируем по датам и SKU
df.sort_values(['timestamp', 'item_id'], inplace=True)
# Индексируем по дате
df.set_index('timestamp', inplace=True)
# Делим на обучающую и проверочные выборки
df_train = df.loc[:'2023-05-01']
df_val = df.loc['2023-05-01':]
# Горизонт прогноза будет равен длине проверочной выборки
pred_len = df_val.index.nunique()
ds_train = new_ds(df_train)
# Обучение
trainer = Trainer(epochs=20, learning_rate=1e-3)
estimator = DeepAREstimator(prediction_length=pred_len, freq=freq,
                            trainer=trainer)
model = estimator.train(ds_train)

<ipython-input-7-e9a9980f2072>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/trainee/Ingate/Ozon/data_ozon.csv',
100%|██████████| 50/50 [00:07<00:00,  6.54it/s, epoch=20/20, avg_epoch_loss=-.761]


In [35]:
# Прогноз на проверочной выборке
ds_val = new_ds(df_val)
preds = list(model.predict(ds_val))
y_pred = np.array([it.median for it in preds])
y_true = np.array([df_val.loc[df_val.item_id == item,
                'target'].to_numpy(float) for item in df_val.item_id.unique()])
res = np.abs(y_pred - y_true).mean(axis=1)

In [38]:
# Средняя абсолютная ошибка (по каждому продукту отдельно)
res.max(), res.min(), res.mean()

(3.093410007854739, 0.16710598694504977, 0.9322785648031641)